In [1]:
import requests
import pandas as pd
import tqdm
from dotenv import load_dotenv
import os
import csv

In [2]:
load_dotenv(dotenv_path='credentials.env')

api_key = os.getenv("API_KEY")
username = os.getenv("USER")

In [14]:
#url = f'https://api-constancias-de-inscripcion.mrbot.com.ar/crear_usuario/?mail={username}'

In [15]:
#res = requests.post(url)
#res.json()

In [3]:
creditos_hist = pd.read_parquet('./input/creditos_hist.parquet')

In [4]:
url = f'https://api-constancias-de-inscripcion.mrbot.com.ar/consulta_constancia_masiva/?usuario={username}&api_key={api_key}'

In [5]:
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}

In [6]:
def resultados_constancias(res):
    datos = {}
    
    datos_generales = res.get('datosGenerales') if res.get('datosGenerales') else None
    datos_regimen = res.get('datosRegimenGeneral') if res.get('datosRegimenGeneral') else None
    error_constancia = res.get('errorConstancia') if res.get('errorConstancia') else None
    error_regimen = res.get('errorRegimenGeneral') if res.get('errorRegimenGeneral') else None
    
    # Datos Generales
    if datos_generales:
        datos['cuit'] = datos_generales.get('idPersona')
    elif error_constancia:
        datos['cuit'] = error_constancia.get('idPersona')
    else:
        datos['cuit'] = None
        
    datos['mes_cierre'] = datos_generales.get('mesCierre') if datos_generales else None
    datos['razon_social'] = datos_generales.get('razonSocial') if datos_generales else None
    datos['contrato_social'] = datos_generales.get('fechaContratoSocial') if datos_generales else None
    datos['tipo_persona'] = datos_generales.get('tipoPersona') if datos_generales else None
    
    # Domicilio Fiscal
    domicilio_fiscal = datos_generales.get('domicilioFiscal') if datos_generales else None
    datos['codigo_postal'] = domicilio_fiscal.get('codPostal') if domicilio_fiscal else None
    datos['provincia'] = domicilio_fiscal.get('descripcionProvincia') if domicilio_fiscal else None
    datos['direccion'] = domicilio_fiscal.get('direccion') if domicilio_fiscal else None
    datos['localidad'] = domicilio_fiscal.get('localidad') if domicilio_fiscal else None
    
    # Datos Régimen
    datos_actividad = datos_regimen.get('actividad', []) if datos_regimen else None
    impuestos = datos_regimen.get('impuesto', []) if datos_regimen else None
    regimenes = datos_regimen.get('regimen', []) if datos_regimen else None
    
    # Error en la constancia
    datos['errorConstancia'] = error_constancia.get('error') if error_constancia else None
    
    # Error en Régimen General
    datos['errorRegimenGeneral'] = error_regimen.get('error') if error_regimen else None
    
    # Actividad
    if datos_actividad:
        datos['actividades'] = '; '.join([
            f"{a.get('descripcionActividad')} ({a.get('idActividad')})" 
            for a in datos_actividad
        ])
        for actividad in datos_actividad:
            id = actividad.get('idActividad')
            datos[f'actividad_{id}'] = True
    
    # Impuestos
    if impuestos:
        datos['impuestos'] = '; '.join([
			f"{i.get('descripcionImpuesto')} ({i.get('idImpuesto')})"
			for i in impuestos
		])
        for impuesto in impuestos:
            id = impuesto.get('idImpuesto')
            datos[f'impuestos_{id}'] = True
    
    # Regímenes
    if regimenes:
        datos['regimenes'] = '; '.join([
            f"{r.get('descripcionRegimen')} ({r.get('idRegimen')})"
			for r in regimenes
		])
        for regimen in regimenes:
            id = regimen.get('idRegimen')
            datos[f'regimen_{id}'] = True
            
    return datos

In [7]:
identificaciones = creditos_hist['identificacion'].unique()
tramos = [identificaciones[i:i + 250] for i in range(0, len(identificaciones), 250)]

In [ ]:
# 189/1343

todos_los_datos = []
session = requests.Session()

for tramo in tqdm.tqdm(tramos):
    try:
        payload = {
        	"cuits": [str(cuit) for cuit in tramo]
        }
        response = session.post(url, json=payload, headers=headers)
        response.raise_for_status()
        todos_los_datos.extend(response.json()['resultado'])
    except Exception as e:
        print(e)

In [9]:
with open('output/constancias.csv', 'w', encoding='utf-8', newline='') as f:
    if todos_los_datos:
        fieldnames = todos_los_datos[0].keys()  
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        
        writer.writeheader()  
        writer.writerows(todos_los_datos)  

In [10]:
datos = []
for respuesta in todos_los_datos:
    datos_procesados = resultados_constancias(respuesta)
    datos.extend([datos_procesados])

datos = pd.DataFrame(datos)

In [11]:
errores = datos[(datos['errorConstancia'].notnull()) | (datos['errorRegimenGeneral'].notnull())]
tipos_errores = []
for error in errores['errorConstancia']:
    if error:
    	tipos_errores.extend(error)
            
for error in errores['errorRegimenGeneral']:
    if error:
        for err in error:
            tipos_errores.extend(error)
            
tipos_errores = list(set(tipos_errores))

with open('./output/errores_constancias.txt', 'w') as f:
    for error in tipos_errores:
        f.write(f'{error}\n')

In [16]:
datos.to_parquet('./input/constancia_inscripcion.parquet', compression= 'gzip')